In [1]:
"""PyTorch based class to read our dataset."""


'PyTorch based class to read our dataset.'

In [2]:
from dataloader import XRayDataset

data = XRayDataset("xray_overview.csv", "..")

from torch.utils.data import random_split
import torch

generator1 = torch.Generator().manual_seed(40)
train_dataset, test_dataset = random_split(data, [0.7, 0.3], generator=generator1)

In [3]:
from torch.utils.data import DataLoader

# DataLoaders prepared
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

In [5]:
from cnn import CNN
from torch import nn

learning_rate = 0.001

model = CNN(3)

# Set Loss function with criterion
criterion = nn.CrossEntropyLoss()

# Set optimizer with optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)


In [7]:
from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter()

In [8]:
def model_train(num_epochs):
    board_idx = 0

    for epoch in range(num_epochs):
        current_loss = 0.
        #Load in the data in batches using the train_loader object
        for i, (images, labels) in enumerate(train_loader):  
            # Move tensors to the configured device
            #images = images.to(device)
            #labels = labels.to(device)
            
            # Forward pass
            outputs = model(images)

            loss = criterion(outputs, labels)
            current_loss += loss.item()
            writer.add_scalar("loss/train", current_loss, board_idx)
            board_idx += 1

            if board_idx % 10 == 0:
                print(f"Mini-batch current-loss: {loss / 10}")
                current_loss = 0.0
                

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
        # Write loss for epoch
        writer.add_scalar("Loss/Epochs", current_loss, epoch)
        print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))


model_train(5)
writer.flush()


Mini-batch current-loss: 0.11401186138391495
Mini-batch current-loss: 0.08949518203735352
Mini-batch current-loss: 0.08152945339679718
Mini-batch current-loss: 0.05598555877804756
Mini-batch current-loss: 0.05834265798330307
Mini-batch current-loss: 0.056603096425533295
Epoch [1/5], Loss: 0.7173
Mini-batch current-loss: 0.06406624615192413
Mini-batch current-loss: 0.07782424241304398
Mini-batch current-loss: 0.05400934815406799
Mini-batch current-loss: 0.06483542919158936
Mini-batch current-loss: 0.06827640533447266
Mini-batch current-loss: 0.053193312138319016
Mini-batch current-loss: 0.04086951166391373
Epoch [2/5], Loss: 0.4087
Mini-batch current-loss: 0.0567978210747242
Mini-batch current-loss: 0.05900583788752556
Mini-batch current-loss: 0.060727037489414215
Mini-batch current-loss: 0.04669959843158722
Mini-batch current-loss: 0.05877018719911575
Mini-batch current-loss: 0.05200041085481644
Epoch [3/5], Loss: 0.5931
Mini-batch current-loss: 0.03857949376106262
Mini-batch current-l

In [9]:
# Evaluate the model
total_correct = 0
total_samples = 0

accuracy = []
losses = []

for i, (image, label) in enumerate(test_loader):

  output = model(image)
  predicted = torch.argmax(output, 1)
  n_label = torch.argmax(label, 1)
  
  # Update the running total of correct predictions and samples
  total_correct += (predicted == n_label).sum().item()
  total_samples += n_label.size(0)

  accuracy.append(100 * total_correct/total_samples)
  losses.append(criterion(output, label))
  print(f"Image of type: {label}")
  print(f"Prediction: {predicted}")
  print("Loss: {:.4f}".format(losses[-1]))
  print(f"Accuracy: {accuracy[-1]}\n")

Image of type: tensor([[1., 0., 0.]])
Prediction: tensor([0])
Loss: 0.1019
Accuracy: 100.0

Image of type: tensor([[1., 0., 0.]])
Prediction: tensor([0])
Loss: 0.0026
Accuracy: 100.0

Image of type: tensor([[0., 1., 0.]])
Prediction: tensor([1])
Loss: 0.2084
Accuracy: 100.0

Image of type: tensor([[0., 0., 1.]])
Prediction: tensor([1])
Loss: 1.1121
Accuracy: 75.0

Image of type: tensor([[1., 0., 0.]])
Prediction: tensor([1])
Loss: 3.6201
Accuracy: 60.0

Image of type: tensor([[0., 1., 0.]])
Prediction: tensor([1])
Loss: 0.0354
Accuracy: 66.66666666666667

Image of type: tensor([[1., 0., 0.]])
Prediction: tensor([0])
Loss: 0.2335
Accuracy: 71.42857142857143

Image of type: tensor([[0., 1., 0.]])
Prediction: tensor([1])
Loss: 0.0751
Accuracy: 75.0

Image of type: tensor([[0., 1., 0.]])
Prediction: tensor([1])
Loss: 0.2235
Accuracy: 77.77777777777777

Image of type: tensor([[1., 0., 0.]])
Prediction: tensor([0])
Loss: 0.0356
Accuracy: 80.0

Image of type: tensor([[0., 0., 1.]])
Prediction

In [ ]:
import matplotlob.pyplot as plt
import numpy as np

plt.plot(np.range(total_samples), accuracy)

In [ ]:
plt.plot(np.range(total_samples), losses)